### This notebook showed that if we try to charge the cell at 100%SOC, PyBaMM will skip that step and give an empty solution. In other software, it may return an error or says the experiment is infeasible.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd

In [2]:
model = pybamm.lithium_ion.DFN()
param = pybamm.ParameterValues("OKane2022")
param.update({"Upper voltage cut-off [V]": 4.21})
param.update({"Lower voltage cut-off [V]": 2.49})
exp = pybamm.Experiment(
    [
        "Hold at 4.2V until C/100", 
        "Rest for 1 hours (5 minute period)", ] )
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 60,  # negative particle
    "r_p": 30,  # positive particle
}

In [3]:
sim1 = pybamm.Simulation(
    model, 
    experiment=exp,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    var_pts=var_pts,
)
solution1 = sim1.solve(calc_esoh=False)

running time: 1.5674792000208981s


the cell is now at 100%SOC, if we want to charge further, it will return an empty solution

In [4]:
exp_2 = pybamm.Experiment([( 
    "Charge at 0.3C until 4.2V", #  (5 minute period) 
    "Hold at 4.2V until C/100 (5 minute period)",
    ),] )
model_2 = model.set_initial_conditions_from(solution1, inplace=True)
sim2 = pybamm.Simulation(
    model_2, 
    experiment=exp_2,
    parameter_values=param,
    solver=pybamm.CasadiSolver(return_solution_if_failed_early=True),
    var_pts=var_pts,
)
sol2 = sim2.solve(calc_esoh=False)

In [6]:
sol2.cycles[0].steps

which means what if we try to acess the result in steps-0, an error will occur

In [7]:
V = sol2.cycles[0].steps[0]["Terminal voltage [V]"].entries

TypeError: 'EmptySolution' object is not subscriptable